In [ ]:
import ROOT as rt
import uproot
import numpy as np
from collections import OrderedDict
import os
import sys
from array import array
sys.path.append(os.getcwd().replace('scripts', 'lib'))

import tdrstyle
a = tdrstyle.setTDRStyle()

print(sys.version)

# load signal ntuples

Load signal ntuples with 4 different LLP lifetimes (0.1, 1, 10, 100m) that contain generator-level LLP information

In [ ]:
fpath =OrderedDict()
tree = OrderedDict()

mass = [40]

OLD_CTAU = np.array([100, 1000, 10000, 100000])#in mm

path = '/eos/uscms/store/user/cmsdas/2024/long_exercises/MDS/signal/nocuts/'

for ct in OLD_CTAU:
    key = 'MC_40_'+str(ct)                       
    fpath[key] = path +  'ggH_HToSSTobbbb_MH-125_MS-40_ctau-'+str(ct)+'_137000pb_weighted.root'

    
NEvents = {}
NEvents_genweight = {}
for k,v in fpath.items():
    root_dir = uproot.open(v) 
    if not root_dir: 
        print(k, "zombie")
        continue
    tree[k] = root_dir['MuonSystem']


# Ex1 - calculate the LLP proper decay length from the LLP four momentum

Here you will calculate the LLP proper decay length by converting from the lab frame decay length to the decay length in the LLP's rest frame, with the following equation:
$d = \beta\gamma\tau$, where d is the lab frame decay length, $\beta$ is the LLP velocity, $\gamma$ is the LLP Lorentz factor, and $\tau$ is the LLP proper decay lifetime.

You will use the following branches in the ROOT tree 
* `gLLP_decay_vertex_x`, in units of cm
* `gLLP_decay_vertex_y`, in units of cm
* `gLLP_decay_vertex_z`, in units of cm
* `gLLP_beta` , ranges from 0 to 1

and you can access a branch by doing `T['branch name'].array()` below, for example to access `gLLP_beta` with `T['gLLP_beta'].array()`. The shape of the LLP-level branches is (number of LLPs, 2), because there are two LLPs per event.

#### Fill in the righthand side of each equation below with the 4 branches mentioned to calculate the proper decay length

In [ ]:
gLLP_ctau = {}
for k, T in tree.items():
    gLLP_decay_vertex = 
    gLLP_gamma = 
    gLLP_ctau[k] = 

# Ex1 - plot the LLP proper decay length

In this exercise, you will plot the `gLLP_ctau` array for 1m and 10m sample.
You will then perform an exponential fit to verify that the shape is exponential and linear in logY scale.

From the exponential fit, you can extract the mean proper decay length of the LLP and verify that it agrees with what you expect from the sample (1m or 10m)

You have to replace the `REPLACE_ME` variable below to fill in the correct variable to extract the fitted ctau by using the variable `fitted_slope`.
Note that the unit in the tree branches are in cm, while the unit in the ROOT file names are in mm

In [ ]:
%%time

Nevents_todraw = 10000
c = rt.TCanvas('c','c', 800, 600)
h = {}

for i, k in enumerate([ 'MC_40_1000', 'MC_40_10000']):
    h[k] = rt.TH1D('', '', 100, 0, 10000)
    h[k].SetXTitle('LLP c#tau [cm]')
    h[k].SetYTitle('fraction of events')
    
    # fill histograms 
    for j in range(Nevents_todraw): h[k].Fill(gLLP_ctau[k][j][0])
    h[k].Scale(1./h[k].Integral()) #normalize histogram to 1
    h[k].SetLineColor(i+1)
    h[k].Draw('hist same')

    ####### perform exponential fit ######
    h[k].Fit('expo')
    fit = h[k].GetFunction('expo')
    fitted_slope = fit.GetParameter(1)
    print("sample: {}, fitted ctau [mm]: {}".format(k, REPLACE_ME))
    #######################################
    
c.SetLogy()
c.Draw()


# Ex2 - reweight LLP lifetime 

In this exercise you will learn how to reweight the LLP lifetime.

In these signal events, there are two LLPs and the decay position of the two LLPs in each event are independent, and each LLP decays with an exponential probability, the distribution of events is simply the product of the two LLP decay probabilities:
\begin{equation}
  p(t_1, t_2 | \tau) = \frac{1}{\tau^2}\exp^{-t_1/\tau}\exp^{-t_2/\tau}
\end{equation}
where $\tau$ is the mean proper decay length of the LLPs, and $t_1$ and $t_2$ are the lifetimes of the first LLP and second LLP in their own rest frame respectively, which are the `gLLP_ctau` variable that you have calculated in Ex1.

To obtain a sample with lifetime $\tau_{new}$ from a sample with lifetime $\tau_{old}$, we
assign a weight, which is the ratio between equation above with parameter $\tau_{new}$
and:
\begin{equation}
  w = (\frac{\tau_{old}}{\tau_{new}})^2 \exp[(t_1+t_2)\times(\frac{1}{\tau_{old}}-\frac{1}{\tau_{new}})]
\end{equation}

You will implement this equation in the function `weight_calc` below.
The probability function for the $\tau_{old}$ has been implemented, you will need to implement the nominator, and calculate the weight by taking tha ratio of the two probability function


In [ ]:
'''
arguements: 
llp_ctau is the list of sum of proper decay length, which is t1+t2 in the equation
new_ctau is tau_new in the equation
old_ctau is tau_old in the equation
All units in cm

return event weight 

'''


def weight_calc(llp_ct, new_ctau, old_ctau):    
    denominator = np.exp(-1.0*llp_ct/old_ctau)/old_ctau**2
    nominator = # FILL IN HERE #
    weight = # FILL IN HERE #
    
    return weight

## reweight from 10m to 1m

In this exercise you will use the reweighting function that you have just implemented to reweight the 10m sample to 1m and plot it against the original 1m sample to validate the reweighting function

You will need to fill in the arguments for the `weight_calc()` function with `llp_ct` and the two lifetimes. Be sure to convert everything to cm.

You will then perform an exponential fit to verify that the extracted mean proper decay length of the LLP is 1m for both by replacing the variable `REPLACE_ME`

In [ ]:
%%time
c = rt.TCanvas('c','c', 800, 600)
h = {}
Nevents_todraw = 10000

for i, k in enumerate(['MC_40_1000', 'MC_40_10000']):
    h[k] = rt.TH1D('', '', 100, 0, 1000)
    h[k].SetXTitle('LLP c#tau [cm]')
    h[k].SetYTitle('fraction of events')
    
    if k == 'MC_40_1000': 
        for j in range(Nevents_todraw): h[k].Fill(gLLP_ctau[k][j][0])
    else: 
        llp_ct = np.sum(gLLP_ctau[k], axis = 1)
        
        # Fill in the arguments in weight_calc()
        weight = weight_calc()
        
        
        for j in range(Nevents_todraw): h[k].Fill(gLLP_ctau[k][j][0], weight[j])
    
    h[k].Scale(1./h[k].Integral())
    h[k].SetLineColor(i+1)
    h[k].Draw('hist same')
    

    ####### perform exponential fit ######
    h[k].Fit('expo')
    fit = h[k].GetFunction('expo')
    fitted_slope = fit.GetParameter(1)
    print("sample: {}, fitted ctau [mm]: {}".format(k, REPLACE_ME))

    #######################################
    
c.SetLogy()
c.Draw()


# Ex3 - Calculate the probability of LLP decaying in the muon system

In this exercise you will calculate the probability of the LLP decaying in the muon system by requiring the LLP decay vertex lies within the Muon system.
You will first define the geometric decay region of the muon system based on Fig. 4.1.1 on page 141 of the [Muon Detector Technical Design Report](https://cds.cern.ch/record/343814?ln=en) by replacing all the `REPLACE_ME` variables below.

Then in the last line you will print the probability of LLP decaying in the endcap muon detectors for LLP mean proper decay lenegths (0.1, 1, 10, 100m) that have been generated.

The probability should be on the order of 15-20% for the 1m and 10m sample

In [ ]:
%%time

for k, T in tree.items():
    
    # define endcap muon system region
    sel_ms = np.abs(T['gLLP_eta'].array()) < REPLACE_ME
    sel_ms = np.logical_and(sel_ms, np.abs(T['gLLP_decay_vertex_z'].array()) < REPLACE_ME)
    sel_ms = np.logical_and(sel_ms, np.abs(T['gLLP_decay_vertex_z'].array()) > REPLACE_ME)
    sel_ms = np.logical_and(sel_ms, np.abs(T['gLLP_decay_vertex_r'].array()) < REPLACE_ME)
    
    print(k, np.count_nonzero(np.sum(sel_ms, axis = 1))/len(sel_ms))

## calulate acceptance vs LLP mean proper decay length

Here we will extend on what we just did to calculate the acceptance for intermediate $c\tau$ by calling the 
reweighting function that you've written in the previous exercise to calculate the probability for other intermediate proper decay lengths.

You will fill in the definition of `sel_ms`, which is what you defined in the cell above and the arguments for `weight_calc()` to reweight to intermediate lifetimes in the `ctau_list`

The list of acceptances for different `ctau` is then saved in the `acceptance` list

In [ ]:
%%time

ctau_list  = [100, 300, 500, 1000, 2000, 5000, 10000, 50000, 100000, 500000] #mm

acceptance = []
for ct in ctau_list:

    # find which sample to use for reweighting
    ct_source = 0
    for i in OLD_CTAU:
        if ct <= i: 
            ct_source = i
            break
    if ct_source == 0: ct_source = OLD_CTAU[-1]
    print(ct, ct_source) #verify the correct sample is chosen
    
    T = tree['MC_40_{}'.format(ct_source)]
    
    ################## FILL IN HERE #################
    sel_ms = # FILL IN HERE
    weight = weight_calc()
    
    #################################################
    
    denominator = np.sum(weight)
    nominator = np.sum(weight[np.sum(sel_ms, axis = 1)>0])
    acceptance.append(nominator/denominator)
    

print(ctau_list, acceptance)

## plot acceptance vs LLP mean proper decay length
here you will plot a `TGraph` with `ctau_list` on the x-axis and `acceptance` on the y-axis
Documentation for `TGraph`: https://root.cern.ch/doc/master/classTGraph.html

Do you understand the shape? Play around with the `ctau_list` to add more points to make the plot smoother!

In [ ]:
%%time

c = rt.TCanvas('c','c', 800, 800)
c.SetRightMargin(0.04)

h = rt.TGraph(len(ctau_list),array("d", np.array(ctau_list)/1000.0), array("d", acceptance))


h.GetXaxis().SetTitle('c#tau [m]')
h.GetYaxis().SetTitle('acceptance')
h.GetXaxis().SetLimits(0.1,1000.0)
h.GetYaxis().SetRangeUser(1e-3,2)

h.GetXaxis().SetTitleOffset(1)
h.GetYaxis().SetTitleSize(0.05)
h.GetYaxis().SetTitleOffset(1.5)

h.SetLineWidth(3)
h.SetLineColor(4)
h.SetMarkerColor(4)

h.Draw('LPA')

c.SetLogy()
c.SetLogx()
c.SetTicky(1)
c.SetTickx(1)


tdrstyle.setTDRStyle()
c.Draw()